In [1]:
# Import Core Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Set Theme for Plotly Maps
# Different MapBox Styles
# carto-darkmatter
# open-street-map
# carto-positron

map_theme = 'carto-darkmatter'

# Set Theme for graphs
sns.set_theme(palette='bright')

# Set Pandas Parameters
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

# Import DataFrame
stl_df = pd.read_csv('../Data/STL-Data-2023.csv')

# Create County DataFrame
stl_county_df = stl_df.loc[stl_df['Geo-County'] == 'Saint Louis County']

# Create City DataFrame
stl_city_df = stl_df.loc[stl_df['Geo-County'] == 'Saint-Louis City']

## Directory Structure

In [2]:
def createAreaDirectories(dataFrame, areaCol, area_path):
    '''
    Title: Create Area Directories
    Description: This will create seperate directories for each area in areaCol.
    dataFrame: The dataframe of data that the area columns are in.
    areaCol: The specfic column of places that need directories created.
    area_path: The path in which you would like the directories created in.
    '''

    # Create list of all area names
    area_list = dataFrame[dataFrame[areaCol].notnull()][areaCol].unique()

    for area in area_list:

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set full path to a variable
        fullPath = os.path.join(area_path,area)

        # Directory boolean
        dirExists = os.path.exists(fullPath)

        # Check if directory already exists
        if dirExists == False:
            # Create directory
            os.mkdir(fullPath)
        else:
            pass

# Create a Folder for each Town
# createAreaDirectories(dataFrame=stl_df,
#                       areaCol='Geo-Town',
#                       area_path='Towns/')

# # Create a Folder for each Village
# createAreaDirectories(dataFrame=stl_df,
#                       areaCol='Geo-Village',
#                       area_path='Villages/')

# # Create a Folder for each Neighborhood
# createAreaDirectories(dataFrame=stl_df,
#                       areaCol='Geo-Neighborhood',
#                       area_path='Neighborhoods/')

## Create Visualizations

### Build Map Cover Images

In [4]:
def area_MapPicture(dataFrame, areaCol, nibrsCol, imagePath):
    '''
    Title: Area Map Picture
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaCol: The column name of the area in which you would to create images for.
    nibrsCol: The column name of the type of nibrs information that would be mapped.
    imagePath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaCol].notnull()][areaCol].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaCol] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsCol].value_counts().reset_index())

        # Create dictionary
        area_vc_mappings = dict()

        # Reverse the order of dataframe
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsCol],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsCol].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{areaCol} Crime Points',   # Chart Name
                             lat=area_latitudes,                               # Latitudes
                             lon=area_longitudes,                              # Longitudes
                             showlegend=False,                                 # Don't show legends
                             marker=dict(color=area_df['NIBRS-Map'])))         # Hover Information Template     

        # Create Layout
        mapPlot.update_layout(height= 900,                                                # Height of Subplot
                              width= 1400,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=12.5,                                      # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        
        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")
        
        # print(area)

        coverImagePath = f'Cover-{area}.jpeg'
        areaPath = f'{imagePath}{area}'

        finalPath = os.path.join(areaPath,coverImagePath)

        # Save image
        mapPlot.write_image(finalPath)

# Create Cover Images for each Town, Village & Neighborhood
# area_MapPicture(dataFrame=stl_df,           # DataFrame
#                 areaCol='Geo-Town',         # Column of Area
#                 nibrsCol='NIBRS-Offense',   # NIBRS Crime Column
#                 imagePath='Towns/')         # Path to directory
# print('Towns are done!')

# area_MapPicture(dataFrame=stl_df,           # DataFrame
#                 areaCol='Geo-Village',      # Column of Area
#                 nibrsCol='NIBRS-Offense',   # NIBRS Crime Column
#                 imagePath='Villages/')      # Path to directory
# print('Villages are done!')

# area_MapPicture(dataFrame=stl_df,             # DataFrame
#                 areaCol='Geo-Neighborhood',   # Column of Area
#                 nibrsCol='NIBRS-Offense',     # NIBRS Crime Column
#                 imagePath='Neighborhoods/')   # Path to directory  
# print('Neighborhoods are done!')

Towns are done!
Villages are done!
Neighborhoods are done!


### Build Density Maps

In [11]:
def area_DensityMap(dataFrame, areaCol, imagePath, zoomLevel):
    '''
    Title: Area Density Map
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaCol: The column name of the area in which you would to create images for.
    nibrsCol: The column name of the type of nibrs information that would be mapped.
    imagePath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaCol].notnull()][areaCol].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaCol] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[['Geo-Latitude', 'Geo-Longitude']].value_counts().reset_index())
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        densityPlot = go.Figure(go.Densitymapbox(name='Crime Density Map',   # Denisty Map Name             
                                                 lat=area_latitudes,         # Latitude values          
                                                 lon=area_longitudes,        # Longitude values       
                                                 radius=10))                 # Radius

        # Create Layout
        densityPlot.update_layout(height= 900,                                              # Height of Subplot
                                width= 1400,                                                # Width of Subplot
                                margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                autosize=True,                                              # Set Autosize of charts to True or False
                                template='plotly_dark',                                     # Template of Subplot
                                mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                            pitch=0,                                        # Pitch of Map
                                            bearing=0,                                      # Bearing of Map
                                            style=map_theme,                                # Theme of Scatter Map
                                            center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                        lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")
        
        # print(area)

        coverImagePath = f'Density-Map-{area}.html'
        areaPath = f'{imagePath}{area}'

        finalPath = os.path.join(areaPath,coverImagePath)

        # Save image
        densityPlot.write_html(finalPath)

# Create a Density Map for each Town, Village & Neighborhood
# area_DensityMap(dataFrame=stl_df,           # DataFrame
#                 areaCol='Geo-Neighborhood',         # Column of Area
#                 imagePath='Neighborhoods/',
#                 zoomLevel = 14)         # Path to directory
# print('Neighborhoods are done!')

# area_DensityMap(dataFrame=stl_df,           # DataFrame
#                 areaCol='Geo-Town',         # Column of Area
#                 imagePath='Towns/',
#                 zoomLevel = 12.5)         # Path to directory
# print('Towns are done!')

# area_DensityMap(dataFrame=stl_df,           # DataFrame
#                 areaCol='Geo-Village',         # Column of Area
#                 imagePath='Villages/',
#                 zoomLevel = 12.5)         # Path to directory
# print('Villages are done!')

Neighborhoods are done!
Towns are done!
Villages are done!


### Build Scatter Maps with Legend by NIBRs Category

In [ ]:
test_df = stl_df[stl_df['Geo-Neighborhood'] == 'Soulard']

figures = px.scatter_mapbox(test_df,
                            lon=test_df['Geo-Longitude'],
                            lat=test_df['Geo-Latitude'],
                            color=test_df['NIBRS-Offense-Category'],
                            labels={""})

figures.update_layout(height= 900,                                              # Height of Subplot
                                width= 1400,                                                # Width of Subplot
                                margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                autosize=True,                                              # Set Autosize of charts to True or False
                                template='plotly_dark',                                     # Template of Subplot
                                legend=dict(title='NIBRs Categories'),
                                mapbox=dict(zoom=14,                                        # Zoom of Map
                                               pitch=0,                                        # Pitch of Map
                                               bearing=0,                                      # Bearing of Map
                                               style=map_theme,                                # Theme of Scatter Map
                                               center=dict(lat=np.mean(test_df['Geo-Latitude']),        # Mean of Area Latitudes
                                                           lon=np.mean(test_df['Geo-Longitude']))))     # Mean of Area Longitudes    # Mean of Area Longitudes

figures.show()

In [ ]:
fig = px.colors.sequential.swatches_continuous() 
fig.show() 

In [111]:
def area_ScatterMap(data_frame, area_col, nibrs_col, imagePath, zoomLevel):

    # Create list of all NeighborHoods
    area_list = data_frame[data_frame[area_col].notnull()][area_col].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = data_frame[data_frame[area_col] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrs_col].value_counts().reset_index())

        area_vc_mappings = dict()

        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrs_col],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrs_col].map(area_vc_mappings)

        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = area_col + ':' + '%{customdata[0]}'
        nibrs_hover = nibrs_col + ':' + '%{customdata[1]}'

        scatterMap = go.Figure(go.Scattermapbox())

        i=0
        markerColors = px.colors.sample_colorscale('plasma_r',samplepoints=area_vc['count']/area_vc['count'].max())

        for nibrsName in area_vc[nibrs_col].unique():

            nibs_df = area_df[area_df[nibrs_col] == nibrsName]
            
            scatterMap.add_trace(go.Scattermapbox(
                visible=True,
                name=nibrsName,
                mode="markers",
                lat=nibs_df['Geo-Latitude'],
                lon=nibs_df['Geo-Longitude'],
                marker=dict(color=f'{markerColors[i]}'),
                customdata=nibs_df[[area_col, nibrs_col]],                # Custom Data Columns for Hover
                hovertemplate="<br>".join([area_hover, nibrs_hover])))
            
            i = i+1

        # Create Layout
        scatterMap.update_layout(height= 900,                                              # Height of Subplot
                                width= 1400,                                                # Width of Subplot
                                clickmode='event',
                                margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                autosize=True,                                              # Set Autosize of charts to True or False
                                template='plotly_dark',                                     # Template of Subplot
                                legend=dict(title=f'{nibrs_col}'),
                                mapbox=dict(zoom=zoomLevel,                                        # Zoom of Map
                                               pitch=0,                                        # Pitch of Map
                                               bearing=0,                                      # Bearing of Map
                                               style=map_theme,                                # Theme of Scatter Map
                                               center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                           lon=np.mean(area_longitudes))))     # Mean of Area Longitudes    # Mean of Area Longitudes
        
        # Add Geographical Data Quality Annotation
        scatterMap.add_annotation(dict(y=0.01,                                             # y position 
                                         x=0.01,                                             # X position 
                                         yref="paper",                                       # set y reference to paper
                                         xref="paper",                                       # Set X reference to paper
                                         xanchor='left',                                     # Set x anchor
                                         showarrow=False,                                    # Don't show arrow
                                         textangle=0,                                        # Angle of Title
                                         font=dict(color='grey',size=8),                    # Font color, size
                                         text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
    

    # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")
        
        # print(area)

        coverImagePath = f'Scatter-Map({nibrs_col})-{area}.html'
        areaPath = f'{imagePath}{area}'

        finalPath = os.path.join(areaPath,coverImagePath)

        # Save image
        scatterMap.write_html(finalPath)

    #scatterMap.show()
# stl_soulard = stl_df[stl_df['Geo-Neighborhood'] == 'Soulard']
# stl_soulard['NIBRS-Offense']

area_ScatterMap(stl_df, 
                'Geo-Neighborhood',
                'NIBRS-Offense',
                'Neighborhoods/', 
                zoomLevel=14)
print('Neighborhoods are done!')

area_ScatterMap(stl_df,           # DataFrame
                'Geo-Town',         # Column of Area
                'NIBRS-Offense',
                'Towns/',
                zoomLevel = 12.5)         # Path to directory
print('Towns are done!')

area_ScatterMap(stl_df,           # DataFrame
                'Geo-Village',         # Column of Area
                'NIBRS-Offense',
                'Villages/',
                zoomLevel = 12.5)  
print('Villages are done!')

Neighborhoods are done!
Towns are done!
Villages are done!


## Create HTML Source code

In [58]:
def stlWebLinks(areaCol, dataFrame, folder_name):
    # Create pandas DF to store names and sort them into alphabetical order
    areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[areaCol].notnull()][areaCol].unique()).sort_values('Name').reset_index(drop=True)
    areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
    areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")

    for i in range(0,len(areaNames)):
        # Open file and set mode to append
        file1 = open("myfile.txt", "a")
    
        # writing newline character
        file1.write(f'''\n<!-- {areaNames['Name'][i]} -->
<a href="google.com">
    <div class="stl-port-box">
        <img src="{folder_name}/{areaNames['TotalFixed'][i]}.jpeg" alt="{areaNames["Name"][i]} Crime Points">
        <div class="stl-port-layer">
            <h4>{areaNames['Name'][i]}</h4>
        </div>
    </div>
</a>''')
    # Close and save file
    file1.close()

stlWebLinks('Geo-Neighborhood', stl_df, 'Neighborhoods')

                Name        SpaceFixed        TotalFixed
0            Academy           Academy           Academy
1              Baden             Baden             Baden
2        Benton Park       Benton-Park       Benton-Park
3   Benton Park West  Benton-Park-West  Benton-Park-West
4          Bevo Mill         Bevo-Mill         Bevo-Mill
..               ...               ...               ...
73  Walnut Park East  Walnut-Park-East  Walnut-Park-East
74  Walnut Park West  Walnut-Park-West  Walnut-Park-West
75  Wells Goodfellow  Wells-Goodfellow  Wells-Goodfellow
76          West End          West-End          West-End
77    Wydown Skinker    Wydown-Skinker    Wydown-Skinker

[78 rows x 3 columns]
